In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install opencv-python-headless

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def extract_frames(video_path, frame_count=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // frame_count

    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (64, 64))
            frames.append(frame)
        else:
            break
    cap.release()
    return np.array(frames)

def load_data(data_dir):
    X = []
    y = []
    labels = {'Defensive Shot': 0, 'Pull Shot': 1, 'Drive Shot': 2, 'Flick Shot': 3}

    for label in labels:
        # print(label)

        videos_dir = os.path.join(data_dir, label)
        videos = os.listdir(videos_dir)
        for video in videos:
            # print(video)
            # print(video.split('.')[0])

            video_path = os.path.join(videos_dir, video)
            frames = extract_frames(video_path)
            if len(frames) == 10:
                X.append(frames)
                y.append(labels[label])

    X = np.array(X)
    y = np.array(y)
    return X, y

def check_data(data_dir, model):
    labels = {'Defensive Shot': 0, 'Pull Shot': 1, 'Drive Shot': 2, 'Flick Shot': 3}

    analysis = {}

    for label in labels:
        videos_dir = os.path.join(data_dir, label)
        videos = os.listdir(videos_dir)

        analysis[label] = {
            'OK': [],
            'NOT OK': []
        }

        for video in videos:
            video_path = os.path.join(videos_dir, video)

            frames = extract_frames(video_path)
            frames = np.expand_dims(frames, axis=0)
            frames = np.expand_dims(frames, axis=-1)

            prediction = model.predict(frames)
            predicted_class = np.argmax(prediction, axis=1)

            # print(prediction)

            if predicted_class[0] == labels[label]:
                analysis[label]['OK'].append(video.split('.')[0])
            else:
                analysis[label]['NOT OK'].append(video.split('.')[0])

    return analysis

In [ ]:
data_dir = '/content/drive/My Drive/ShotDetection'
X, y = load_data(data_dir)

In [ ]:
input_shape = X.shape
input_shape

(1278, 10, 64, 64, 3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_val = X_val / 255.0

In [ ]:
input_shape = X_train.shape[1:]
input_shape

(10, 64, 64, 3)

In [ ]:
# (batch_size, time_steps, height, width, channels)
X_train = np.reshape(X_train, (X_train.shape[0], 10, 64, 64, 3))
X_val = np.reshape(X_val, (X_val.shape[0], 10, 64, 64, 3))

In [ ]:
input_shape = X_train.shape[1:]
input_shape

(10, 64, 64, 3)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout

def create_LRCN_model(input_shape):
    model = Sequential()

    # Layer 1
    model.add(TimeDistributed(Conv2D(filters=16, kernel_size=(3, 3), activation='tanh', padding='same'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.2)))

    # Layer 2
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='tanh', padding='same')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.2)))

    # Layer 3
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='tanh', padding='same')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(TimeDistributed(MaxPooling2D((2, 2))))

    # Flatten the output
    model.add(TimeDistributed(Flatten()))

    # Dense Layer
   # model.add(TimeDistributed(Dense(64, activation='relu')))

    # LSTM Layer
    model.add(LSTM(32))

    # Output Layer
    model.add(Dense(4, activation='softmax'))  # Assuming 4 classes for the output

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# input_shape = X_train.shape[1:]
# print(input_shape)
# model = create_LRCN_model(input_shape)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_44 (TimeD  (None, 10, 64, 64, 16)    448       
 istributed)                                                     
                                                                 
 time_distributed_45 (TimeD  (None, 10, 16, 16, 16)    0         
 istributed)                                                     
                                                                 
 time_distributed_46 (TimeD  (None, 10, 16, 16, 16)    0         
 istributed)                                                     
                                                                 
 time_distributed_47 (TimeD  (None, 10, 16, 16, 32)    4640      
 istributed)                                                     
                                                                 
 time_distributed_48 (TimeD  (None, 10, 4, 4, 32)     

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/30
64/64 [==============================] - 33s 476ms/step - loss: 1.3918 - accuracy: 0.2598 - val_loss: 1.3828 - val_accuracy: 0.2607
Epoch 2/30
64/64 [==============================] - 28s 442ms/step - loss: 1.3752 - accuracy: 0.2812 - val_loss: 1.3970 - val_accuracy: 0.2996
Epoch 3/30
64/64 [==============================] - 29s 459ms/step - loss: 1.3727 - accuracy: 0.3008 - val_loss: 1.3532 - val_accuracy: 0.3541
Epoch 4/30
64/64 [==============================] - 29s 452ms/step - loss: 1.3385 - accuracy: 0.3633 - val_loss: 1.2963 - val_accuracy: 0.4436
Epoch 5/30
64/64 [==============================] - 26s 410ms/step - loss: 1.3295 - accuracy: 0.3564 - val_loss: 1.2455 - val_accuracy: 0.4514
Epoch 6/30
64/64 [==============================] - 28s 432ms/step - loss: 1.2308 - accuracy: 0.4346 - val_loss: 1.1492 - val_accuracy: 0.5214
Epoch 7/30
64/64 [==============================] - 28s 443ms/step - loss: 1.1035 - accuracy: 0.5371 - val_loss: 1.0170 - val_accuracy: 0.6109

In [ ]:
model.save('/content/drive/My Drive/ShotDetection/LRCN_model.h5')

NameError: name 'model' is not defined

In [ ]:
model_path = '/content/drive/My Drive/ShotDetection/LRCN_model.h5'

# Load the model
model = tf.keras.models.load_model(model_path)

In [ ]:
from google.colab import files

uploaded = files.upload()

# Move the uploaded video to a specific directory
os.makedirs('videos', exist_ok=True)
for file_name in uploaded.keys():
    os.rename(file_name, os.path.join('videos', file_name))

Saving 303.mp4 to 303.mp4


In [ ]:
video_path = 'videos/303.mp4'
new_frames = extract_frames(video_path)
new_frames = np.expand_dims(new_frames, axis=0)
new_frames = np.expand_dims(new_frames, axis=-1)

prediction = model.predict(new_frames)
predicted_class = np.argmax(prediction, axis=1)

print(prediction)
shot = {0: 'Defensive Shot', 1: 'Pull Shot', 2: 'Drive Shot', 3: 'Flick Shot'}
print(shot[predicted_class[0]])

# Defensive | OK     - 253, 254, 300, 129
#           | NOT OK - 234, 211, 84, 137, 303

1/1 [==============================] - 0s 27ms/step
[[0.25894135 0.00087355 0.6204986  0.11968652]]
Drive Shot


In [ ]:
data_dir = '/content/drive/My Drive/ShotDetection'
analysis = check_data(data_dir, model)

1/1 [==============================] - 0s 45ms/step
[[0.01414547 0.00749732 0.9740432  0.00431406]]
1/1 [==============================] - 0s 59ms/step
[[0.96046907 0.00300245 0.02384111 0.01268738]]
1/1 [==============================] - 0s 28ms/step
[[0.8260243  0.00944078 0.15233104 0.01220386]]
1/1 [==============================] - 0s 41ms/step
[[9.6736044e-01 6.1391375e-04 2.2891087e-02 9.1345822e-03]]
1/1 [==============================] - 0s 29ms/step
[[0.0023655  0.9866232  0.00756376 0.00344747]]
1/1 [==============================] - 0s 28ms/step
[[0.25400275 0.51212    0.13280617 0.10107104]]
1/1 [==============================] - 0s 29ms/step
[[0.09673684 0.05245732 0.8434145  0.00739145]]
1/1 [==============================] - 0s 30ms/step
[[0.5560007  0.03899454 0.36489025 0.04011457]]
1/1 [==============================] - 0s 27ms/step
[[0.38021868 0.10596025 0.49111938 0.0227017 ]]
1/1 [==============================] - 0s 32ms/step
[[0.01070444 0.8033917  0.1821369  0

KeyboardInterrupt: 

In [ ]:
analysis

{'Defensive Shot': {'OK': ['45',
   '316',
   '275',
   '47',
   '281',
   '245',
   '260',
   '285',
   '44',
   '306',
   '247',
   '40',
   '239',
   '257',
   '237',
   '43',
   '42',
   '263',
   '68',
   '66',
   '67',
   '15',
   '59',
   '283',
   '56',
   '274',
   '140',
   '235',
   '36',
   '325',
   '9',
   '264',
   '213',
   '280',
   '208',
   '218',
   '219',
   '217',
   '214',
   '172',
   '191',
   '171',
   '187',
   '190',
   '184',
   '188',
   '141',
   '177',
   '161',
   '254',
   '181',
   '147',
   '301',
   '174',
   '154',
   '162',
   '163',
   '284',
   '133',
   '128',
   '304',
   '300',
   '134',
   '135',
   '319',
   '129',
   '91',
   '99',
   '96'],
  'NOT OK': ['318',
   '278',
   '31',
   '236',
   '46',
   '324',
   '276',
   '267',
   '261',
   '241',
   '282',
   '39',
   '243',
   '288',
   '262',
   '41',
   '323',
   '139',
   '64',
   '62',
   '22',
   '16',
   '61',
   '34',
   '238',
   '21',
   '60',
   '55',
   '53',
   '48',
   '57',